In [3]:
import pandas as pd
good = pd.read_csv('good.tsv', sep='\t')
good.head()

context_2  \
0  кликни на меня а потом на надпись " видео - зв...   
1  кликни на меня а потом на надпись " видео - зв...   
2  кликни на меня а потом на надпись " видео - зв...   
3  кликни на меня а потом на надпись " видео - зв...   
4                                 бывало и получше .   

                                          context_1  \
0                                 о , я тебя вижу .   
1                                 о , я тебя вижу .   
2                                 о , я тебя вижу .   
3                                 о , я тебя вижу .   
4  слушайте , мы с женой . . . совсем заблудились .   

                                           context_0  \
0                            ладно , повесь трубку .   
1                            ладно , повесь трубку .   
2                            ладно , повесь трубку .   
3                            ладно , повесь трубку .   
4  я пытаюсь добраться до юджина , но , кажется ,...   

                        reply    label  
0                   не могу .     good  
1      нет , звонить буду я .  neutral  
2     я не прекращу звонить .     good  
3                я звоню им .     good  
4  куда же вы едете , месье ?     good

In [6]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer()

vectorizer.fit(good.context_0)

matrix_big = vectorizer.transform(good.context_0)

matrix_big.shape

(60049, 14123)

In [7]:
from sklearn.decomposition import TruncatedSVD

svd = TruncatedSVD(n_components=300)

svd.fit(matrix_big)
matrix_small = svd.transform(matrix_big)
matrix_small.shape

(60049, 300)

In [8]:
import numpy as np
from sklearn.neighbors import BallTree
from sklearn.base import BaseEstimator

def softmax(x):
    proba = np.exp(-x)
    return proba/sum(proba)

class NeighborSampler(BaseEstimator):
    def __init__(self, k=5, temp=1.0):
        self.k = k
        self.temp = temp
    
    def fit(self, X, y):
        self.tree_ = BallTree(X)
        self.y_ = np.array(y)
    
    def predict(self, X, random_state=None):
        distances, indeces = self.tree_.query(X, return_distance=True, k=self.k)
        result = []
        for distance, index in zip(distances, indeces):
            result.append(np.random.choice(index, p=softmax(distance*self.temp)))
        return self.y_[result]

In [9]:
from sklearn.pipeline import make_pipeline
ns = NeighborSampler()
ns.fit(matrix_small, good.reply)
pipe = make_pipeline(vectorizer, svd, ns)

In [10]:
print(pipe.predict(['Как дела?']))

['как сажа бела .']


In [12]:
import joblib
filename = 'speak_bot_vk.joblib.pkl'
_ = joblib.dump(pipe, filename)